# Improving energy estimation of a Fermionic Hamiltonian with SQD

https://qiskit.github.io/qiskit-addon-sqd/tutorials/01_chemistry_hamiltonian.html

In [1]:
import warnings

warnings.filterwarnings("ignore")

import pyscf
import pyscf.cc
import pyscf.mcscf


ModuleNotFoundError: No module named 'pyscf'